In [42]:
import kfp
import kfp.components as comp
import kfp.compiler as compiler
import kfp.dsl as dsl

# Init an experiment

In [43]:
EXPERIMENT_NAME = 'SLEQ_BASICS'
PIPELINE_FILENAME = 'simple_addition.tar.gz'
PIPELINE_RUN_NAME = '[basic] simple addition'

In [44]:
client = kfp.Client()

existing_experiments = client.list_experiments().experiments

if existing_experiments is not None:
    experiment = next(iter([exp for exp in existing_experiments if exp.name == EXPERIMENT_NAME]), None)
else:
    experiment = None
    
if experiment is None:
    experiment = client.create_experiment(EXPERIMENT_NAME)
    print('Experiment %s created with ID %s' % (experiment.name, experiment.id))
else:
    print('Experiment already exists with id %s' % experiment.id)

Experiment already exists with id 57935e68-839a-492e-8140-98fcb83a10d0


# Defines simple pipeline

In [45]:
def add(a: float,  b: float) -> float:
    print('adding %s and %s' % (a, b))
    return a + b

add_operation = comp.func_to_container_op(add)

In [46]:
@dsl.pipeline(
    name='simple_pipeline',
    description='my first pipeline'
)
def pipeline(a: float, b: float, c: float) -> float:
    add_task_1 = add_operation(a, b)
    add_task_2 = add_operation(add_task_1.output, c)

pipeline_func = compiler.Compiler().compile(pipeline, PIPELINE_FILENAME)

# Running the pipeline

In [47]:
arguments = { 'a': 2, 'b': 10, 'c': 13.25 }

client.run_pipeline(experiment.id, PIPELINE_RUN_NAME, PIPELINE_FILENAME, arguments)

{'created_at': datetime.datetime(2019, 3, 11, 18, 16, 59, tzinfo=tzlocal()),
 'description': None,
 'error': None,
 'id': 'dc90b562-4429-11e9-8c13-080027606df2',
 'metrics': None,
 'name': 'simple run',
 'pipeline_spec': {'parameters': [{'name': 'a', 'value': '2'},
                                  {'name': 'b', 'value': '10'},
                                  {'name': 'c', 'value': '13.25'}],
                   'pipeline_id': None,
                   'pipeline_manifest': None,
                   'workflow_manifest': '{"apiVersion": '
                                        '"argoproj.io/v1alpha1", "kind": '
                                        '"Workflow", "metadata": '
                                        '{"generateName": "simple-pipeline-"}, '
                                        '"spec": {"arguments": {"parameters": '
                                        '[{"name": "a"}, {"name": "b"}, '
                                        '{"name": "c"}]}, "entrypoint": '
       